# MLOps Coloring Book

This notebook may be used for demonstration of AutoMLOps


# Install

In [1]:
!pip3 install ../dist/AutoMLOps-1.0.0-py2.py3-none-any.whl --user

Processing /home/jupyter/1-ClickMLOps/dist/AutoMLOps-1.0.0-py2.py3-none-any.whl


Restart the kernel after installing the package

# Upload Data

In [17]:
!python -m data.load_data_to_bq --project automlops-sandbox --file data/Dry_Beans_Dataset.csv

Dataset automlops-sandbox.test_dataset already exists
Table test_dataset.dry-beans already exists


## 1. Without Using KFP Spec
This workflow will generate a pipeline without using Kubeflow spec. `generate()` will create all the necessary files but not run them. `run()` will create all the necessary files, resources, and then push the code to the source repo to trigger the build. Please view the readme for more information.

## Imports

In [7]:
from AutoMLOps import AutoMLOps

In [8]:
%%define_imports
import json
import pandas as pd
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
from google.cloud import bigquery
from google.cloud import storage
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from joblib import dump
import pickle
import os

## Data Loading

In [9]:
%%define_component
AutoMLOps.makeComponent(
    name="create_dataset",
    description="Loads data from BQ and writes a dataframe as a csv to GCS.", # optional
    params=[
        {"name": "bq_table", "type": str}, # descriptions are optional
        {"name": "data_path", "type": str, "description": "GS location where the training data is written."},
        {"name": "project_id", "type": str, "description": "Project_id."}
    ]
)
# Component code goes below:
bq_client = bigquery.Client(project=project_id)

def get_query(bq_input_table: str) -> str:
    """Generates BQ Query to read data.

    Args:
    bq_input_table: The full name of the bq input table to be read into
    the dataframe (e.g. <project>.<dataset>.<table>)
    Returns: A BQ query string.
    """
    return f"""
    SELECT *
    FROM `{bq_input_table}`
    """

def load_bq_data(query: str, client: bigquery.Client) -> pd.DataFrame:
    """Loads data from bq into a Pandas Dataframe for EDA.
    Args:
    query: BQ Query to generate data.
    client: BQ Client used to execute query.
    Returns:
    pd.DataFrame: A dataframe with the requested data.
    """
    df = client.query(query).to_dataframe()
    return df

dataframe = load_bq_data(get_query(bq_table), bq_client)
dataframe.to_csv(data_path)

## Model Training

In [10]:
%%define_component
AutoMLOps.makeComponent(
    name="train_model",
    description="Trains a decision tree on the training data.",
    params=[
        {"name": "model_directory", "type": str, "description": "GS location of saved model."},
        {"name": "data_path", "type": str, "description": "GS location where the training data."}
    ]
)
# Component code goes below:
def save_model(model, model_directory):
    """Saves a model to uri."""
    filename = f'model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    
    bucket_name = model_directory.split('/')[2]
    prefix='/'.join(model_directory.split('/')[3:])
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(os.path.join(prefix, filename))
    blob.upload_from_filename(filename)

df = pd.read_csv(data_path)
labels = df.pop("Class").tolist()
data = df.values.tolist()
x_train, x_test, y_train, y_test = train_test_split(data, labels)
skmodel = DecisionTreeClassifier()
skmodel.fit(x_train,y_train)
score = skmodel.score(x_test,y_test)
print('accuracy is:',score)

output_uri = os.path.join(model_directory, f'model.pkl')
save_model(skmodel, model_directory)

## Uploading & Deploying the Model

In [11]:
%%define_component
AutoMLOps.makeComponent(
    name="deploy_model",
    description="Trains a decision tree on the training data.",
    params=[
        {"name": "model_directory", "type": str, "description": "GS location of saved model."},
        {"name": "project_id", "type": str, "description": "Project_id."},
        {"name": "region", "type": str, "description": "Region."}
    ]
)
# Component code goes below:
aiplatform.init(project=project_id, location=region)
deployed_model = aiplatform.Model.upload(
    display_name="beans-model-pipeline",
    artifact_uri = model_directory,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
)
endpoint = deployed_model.deploy(machine_type="n1-standard-4")

## Define and Run the Pipeline

In [12]:
AutoMLOps.makePipeline(
    name="training-pipeline",
    description="description", # optional
    params=[
        {"name": "bq_table", "type": str}, # descriptions are optional
        {"name": "model_directory", "type": str, "description": "Description."},
        {"name": "data_path", "type": str, "description": "Description."},
        {"name": "project_id", "type": str, "description": "Description."},
        {"name": "region", "type": str, "description": "Description."}
    ],
    pipeline=[{
        "component_name": "create_dataset", "param_mapping": [
            ("bq_table", "bq_table"), # (component_param, pipeline_param)
            ("data_path", "data_path"),
            ("project_id", "project_id")
        ]
    },
    {
        "component_name": "train_model", "param_mapping": [
            ("model_directory", "model_directory"),
            ("data_path", "data_path")
        ]
    },
    {
        "component_name": "deploy_model", "param_mapping": [
            ("model_directory", "model_directory"),
            ("project_id", "project_id"),
            ("region", "region")
        ]
    }]
)

In [13]:
PROJECT_ID = "automlops-sandbox"
pipeline_params = {
    "bq_table": f"{PROJECT_ID}.test_dataset.dry-beans",
    "model_directory": f"gs://{PROJECT_ID}-bucket/trained_models/{datetime.datetime.now()}",
    "data_path": "gs://{PROJECT_ID}-bucket/data",
    "project_id": f"{PROJECT_ID}",
    "region": "us-central1"
}

In [14]:
AutoMLOps.generate(project_id=f'{PROJECT_ID}', pipeline_params=pipeline_params, use_kfp_spec=False, run_local=False, schedule='0 */12 * * *')

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt


In [15]:
AutoMLOps.go(project_id=f'{PROJECT_ID}', pipeline_params=pipeline_params, use_kfp_spec=False, run_local=False, schedule='0 */12 * * *')

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt
Updating required API services in project automlops-sandbox
Operation "operations/acat.p2-45373616427-b89d0914-73cc-44e2-a6ea-40c3412be6ae" finished successfully.
Listing items under project automlops-sandbox, location us-central1.

vertex-mlops-af  DOCKER  STANDARD_REPOSITORY  Artifact Registry vertex-mlops-af in us-central1.  us-central1          Google-managed key  2023-01-11T22:12:26  2023-01-11T22:22:22  521.264
Artifact Registry: vertex-mlops-af already exists in project automlops-sandbox
gs://automlops-sandbox-bucket/
GS Bucket: automlops-sandbox-bucket already exists in project automlops-sandbox
Pipeline Runner Service Account         vertex-pipelines@automlops-sandbox.iam.gserviceaccount.com  False
Service Account: vertex-pipelines already exists in project automlops-sandbox
Updating required IAM roles in project automlops-sandbox
AutoMLOps-repo  automlops-sandbox  https://source

remote: It seems you're using an old version of Git (git/2.25.1,gzip(gfe),gzip(gfe)). Please update the client following the instructions in go/old-git-client        
To https://source.developers.google.com/p/automlops-sandbox/r/AutoMLOps-repo
   9d8317a..3734d16  master -> master


Pushing code to main branch, triggering cloudbuild...
Waiting for cloudbuild job to complete.................Submitting PipelineJob...
Submitting training job to Cloud Runner Service https://run-pipeline-q5owjmymra-uc.a.run.app in project  using @pipelines/runtime_parameters/pipeline_parameter_values.json
Note: Unnecessary use of -X or --request, POST is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0== Info:   Trying 216.239.38.53:443...
== Info: TCP_NODELAY set
== Info: Connected to run-pipeline-q5owjmymra-uc.a.run.app (216.239.38.53) port 443 (#0)
== Info: ALPN, offering http/1.1
== Info: successfully set certificate verify locations:
== Info:   CAfile: /etc/ssl/certs/ca-certificates.crt
  CApath: /etc/ssl/certs
=> Send SSL data, 5 bytes (0x5)
0000: .....
== Info: TLSv1.3 (OUT), 

## 2. Using KFP Spec

## Imports

In [ ]:
import json
import pandas as pd
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
import datetime

from AutoMLOps import AutoMLOps

## Data Loading

In [ ]:
@component(
    packages_to_install=[
        "google-cloud-bigquery", 
        "pandas",
        "pyarrow",
        "db_dtypes"
    ],
    base_image="python:3.9",
    output_component_file=f"{AutoMLOps.OUTPUT_DIR}/create_dataset.yaml"
)
def create_dataset(
    bq_table: str,
    output_data_path: OutputPath("Dataset"),
    project: str
):
    from google.cloud import bigquery
    import pandas as pd
    bq_client = bigquery.Client(project=project)


    def get_query(bq_input_table: str) -> str:
        """Generates BQ Query to read data.

        Args:
        bq_input_table: The full name of the bq input table to be read into
        the dataframe (e.g. <project>.<dataset>.<table>)
        Returns: A BQ query string.
        """
        return f"""
        SELECT *
        FROM `{bq_input_table}`
        """

    def load_bq_data(query: str, client: bigquery.Client) -> pd.DataFrame:
        """Loads data from bq into a Pandas Dataframe for EDA.
        Args:
        query: BQ Query to generate data.
        client: BQ Client used to execute query.
        Returns:
        pd.DataFrame: A dataframe with the requested data.
        """
        df = client.query(query).to_dataframe()
        return df

    dataframe = load_bq_data(get_query(bq_table), bq_client)
    dataframe.to_csv(output_data_path)

## Model Training

In [ ]:
@component(
    packages_to_install=[
        "scikit-learn",
        "pandas",
        "joblib",
        "tensorflow"
    ],
    base_image="python:3.9",
    output_component_file=f"{AutoMLOps.OUTPUT_DIR}/train_model.yaml",
)
def train_model(
    output_model_directory: str,
    dataset: Input[Dataset],
    metrics: Output[Metrics],
    model: Output[Model]
):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import train_test_split
    from joblib import dump
    import pandas as pd
    import tensorflow as tf
    import pickle
    import os
    
    def save_model(model, uri):
        """Saves a model to uri."""
        with tf.io.gfile.GFile(uri, 'w') as f:
            pickle.dump(model, f)
    
    df = pd.read_csv(dataset.path)
    labels = df.pop("Class").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)
    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train,y_train)
    score = skmodel.score(x_test,y_test)
    print('accuracy is:',score)
    metrics.log_metric("accuracy",(score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))

    output_uri = os.path.join(output_model_directory, f'model.pkl')
    save_model(skmodel, output_uri)
    model.path = output_model_directory

## Uploading & Deploying the Model

In [ ]:
@component(
    packages_to_install=[
        "google-cloud-aiplatform"
    ],
    base_image="python:3.9",
    output_component_file=f"{AutoMLOps.OUTPUT_DIR}/deploy_model.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform
    aiplatform.init(project=project, location=region)
    deployed_model = aiplatform.Model.upload(
        display_name="beans-model-pipeline",
        artifact_uri = model.uri,
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

## Define and Run the Pipeline

In [ ]:
%%define_kfp_pipeline

@dsl.pipeline(name='training-pipeline')
def pipeline(bq_table: str,
             output_model_directory: str,
             project: str,
             region: str,
            ):

    dataset_task = create_dataset(
        bq_table=bq_table, 
        project=project)

    model_task = train_model(
        output_model_directory=output_model_directory,
        dataset=dataset_task.output)

    deploy_task = deploy_model(
        model=model_task.outputs["model"],
        project=project,
        region=region)


In [ ]:
pipeline_params = {
    "bq_table": "sandbox-srastatter.mlops_boxer_test.dry-beans",
    "output_model_directory": f"gs://mlops-boxer-test/trained_models/{datetime.datetime.now()}",
    "project": "sandbox-srastatter",
    "region": "us-central1"
}

In [ ]:
AutoMLOps.go(project_id='sandbox-srastatter', pipeline_params=pipeline_params, use_kfp_spec=True, run_local=False, schedule='0 */12 * * *')

Listing items under project sandbox-srastatter, location us-central1.

vertex-mlops-af                DOCKER  STANDARD_REPOSITORY               us-central1          Google-managed key  2022-05-04T16:27:05  2023-01-04T17:45:11  16576.770
Artifact Registry: vertex-mlops-af already exists in project sandbox-srastatter
gs://sandbox-srastatter-bucket/
GS Bucket: sandbox-srastatter-bucket already exists in project sandbox-srastatter
vertex-pipelines                        vertex-pipelines@sandbox-srastatter.iam.gserviceaccount.com  False
Service Account: vertex-pipelines already exists in project sandbox-srastatter
Updated IAM policy for project [sandbox-srastatter].
bindings:
- members:
  - serviceAccount:service-1006819402307@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1006819402307@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:1006819402307

remote: It seems you're using Apple Git (git/2.37.1 (Apple Git-137.1),gzip(gfe),gzip(gfe)). Apple Git is not frequently updated and often has known vulnerabilities. Please follow the instructions at go/old-git-client#gmac to use a more current version of Git.        
To https://source.developers.google.com/p/sandbox-srastatter/r/AutoMLOps-repo
   28dca70..acb8d7d  main -> main


Pushing code to main branch, triggering cloudbuild...
Waiting for cloudbuild job to complete...................................Submitting PipelineJob...
Note: Unnecessary use of -X or --request, POST is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0== Info:   Trying 216.239.32.53:443...
== Info: Connected to run-pipeline-wxma5l6a2a-uc.a.run.app (216.239.32.53) port 443 (#0)
== Info: ALPN: offers http/1.1
== Info:  CAfile: /etc/ssl/cert.pem
== Info:  CApath: none
== Info: (304) (OUT), TLS handshake, Client hello (1):
=> Send SSL data, 338 bytes (0x152)
0000: ...N..y.E}.qi.(C.+%..!NQW.......E)...u p..wV..g.?...c....9...9fE
0040: f$...w..b.............0.,.(.$.......k.9...........=.5...../.+.'.
0080: #.......g.3...E...<./...A.......................+............3.&
00c0: .$... .}i.=.-..